In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D, Conv1D, MaxPooling1D, Embedding, Dropout
from tensorflow.keras.models import Model
from keras import regularizers

In [14]:
base = pd.read_csv("spam.csv", encoding="ISO-8859-1")
base.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [15]:
base = base.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
base.columns = ["labels", "data"]

In [16]:
base.head()

,labels,data
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [17]:
base["b_labels"] = base["labels"].map({"ham": 0, "spam": 1})

y = base["b_labels"].values
x = base["data"]

In [18]:
x_treinamento , x_teste, y_trainamento, y_teste = train_test_split(x, y, test_size=0.33)

In [19]:
tokenizer = Tokenizer(num_words=20000)

tokenizer.fit_on_texts(x_treinamento)

sequences_trainamento = tokenizer.texts_to_sequences(x_treinamento)
sequences_teste = tokenizer.texts_to_sequences(x_teste)

In [28]:
V = len(tokenizer.word_index)
max_len_train = max(len(seq) for seq in sequences_trainamento)

T = max_len_train

data_treinamento = pad_sequences(sequences_trainamento, maxlen=T)
data_teste = pad_sequences(sequences_teste, maxlen=T)

In [21]:
D = 20
i = Input(shape=(T, ))

layer = Embedding(V + 1, D)(i)
layer = Conv1D(32, 3, activation='relu')(layer)
layer = MaxPooling1D(3)(layer)


layer = Conv1D(64, 3, activation='relu')(layer)
layer = MaxPooling1D(3)(layer)

layer = Conv1D(128, 3, activation='relu')(layer)
layer = GlobalMaxPooling1D()(layer)

layer = Dropout(0.2)(layer)
layer = Dense(64, kernel_regularizer=regularizers.l1(0.01), activation='relu')(layer)

layer = Dense(1, activation='sigmoid')(layer)

model = Model(i, layer)

In [29]:
model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

results = model.fit(data_treinamento , y_trainamento, epochs=20, validation_data=(data_teste, y_teste))

Epoch 1/20


117/117 [==============================] - 8s 22ms/step - loss: 4.0180 - accuracy: 0.8674 - val_loss: 1.2952 - val_accuracy: 0.8630
Epoch 2/20
117/117 [==============================] - 2s 14ms/step - loss: 0.4887 - accuracy: 0.8741 - val_loss: 0.2726 - val_accuracy: 0.9201
Epoch 3/20
117/117 [==============================] - 1s 10ms/step - loss: 0.2009 - accuracy: 0.9585 - val_loss: 0.2101 - val_accuracy: 0.9603
Epoch 4/20
117/117 [==============================] - 1s 12ms/step - loss: 0.0920 - accuracy: 0.9877 - val_loss: 0.1632 - val_accuracy: 0.9772
Epoch 5/20
117/117 [==============================] - 1s 12ms/step - loss: 0.0635 - accuracy: 0.9962 - val_loss: 0.1098 - val_accuracy: 0.9859
Epoch 6/20
117/117 [==============================] - 1s 12ms/step - loss: 0.0425 - accuracy: 0.9989 - val_loss: 0.0843 - val_accuracy: 0.9859
Epoch 7/20
117/117 [==============================] - 1s 11ms/step - loss: 0.0383 - accuracy: 0.9992 - val_loss: 0.0832 - val_accuracy: 0.98